In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
control = pd.read_csv('control_group.csv', sep=';')
test = pd.read_csv('test_group.csv', sep=';')

In [9]:
control = control.dropna()
test = test.dropna()

control.columns = control.columns.str.replace('# of', '')
control.columns = control.columns.str.replace(' [USD]', '')
control.columns = control.columns.str.replace(' ', '')

test.columns = test.columns.str.replace('# of', '')
test.columns = test.columns.str.replace(' [USD]', '')
test.columns = test.columns.str.replace(' ', '')

control.head()

,CampaignName,Date,Spend,Impressions,Reach,WebsiteClicks,Searches,ViewContent,AddtoCart,Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
5,Control Campaign,6.08.2019,3083,109076.0,87998.0,4028.0,1709.0,1249.0,784.0,764.0


In [7]:
control.describe()

,Spend,Impressions,Reach,WebsiteClicks,Searches,ViewContent,AddtoCart,Purchase
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,2304.068966,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.000000,522.793103
std,363.534822,21688.922908,21832.349595,1757.369003,866.089368,777.545469,407.457973,185.028642
min,1757.000000,71274.000000,42859.000000,2277.000000,1001.000000,848.000000,442.000000,222.000000
25%,1962.000000,92029.000000,74192.000000,4085.000000,1615.000000,1249.000000,930.000000,372.000000
50%,2319.000000,113430.000000,91579.000000,5224.000000,2390.000000,1984.000000,1339.000000,501.000000
75%,2544.000000,121332.000000,102479.000000,6628.000000,2711.000000,2421.000000,1641.000000,670.000000
max,3083.000000,145248.000000,127852.000000,8137.000000,4891.000000,4219.000000,1913.000000,800.000000


In [8]:
test.describe()

,Spend,Impressions,Reach,WebsiteClicks,Searches,ViewContent,AddtoCart,Purchase
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2563.066667,74584.800000,53491.566667,6032.333333,2418.966667,1858.000000,881.533333,521.233333
std,348.687681,32121.377422,28795.775752,1708.567263,388.742312,597.654669,347.584248,211.047745
min,1968.000000,22521.000000,10598.000000,3038.000000,1854.000000,858.000000,278.000000,238.000000
25%,2324.500000,47541.250000,31516.250000,4407.000000,2043.000000,1320.000000,582.500000,298.000000
50%,2584.000000,68853.500000,44219.500000,6242.500000,2395.500000,1881.000000,974.000000,500.000000
75%,2836.250000,99500.000000,78778.750000,7604.750000,2801.250000,2412.000000,1148.500000,701.000000
max,3112.000000,133771.000000,109834.000000,8264.000000,2978.000000,2801.000000,1391.000000,890.000000


<h6> Key metrics: Purchase<h6>

In [11]:
from scipy.stats import shapiro
alpha = 0.05

_, pval = shapiro(control['Purchase'])
pval

0.08964364230632782

In [12]:
_, pval = shapiro(test['Purchase'])
pval

0.024077769368886948

Not from normal distribution, so we need to use Mann-Whitney U Test

In [14]:
from scipy.stats import mannwhitneyu

_, pval = mannwhitneyu(control['Purchase'], test['Purchase'])
pval

0.957674926453464

So there is no big diff in Purchase between groups